In [ ]:
# !rm -r /home/karem/DatasetKaggleCovid/Dataset/Original/.ipynb_checkpoints
# !rm -fr /home/karem/.local/share/Trash/files/*

In [15]:
import pandas as pd
import os
import numpy as np

from pathlib import Path

In [16]:
diretorio = os.getcwd() + '/'
print(diretorio)

#############################
# Escolhe se vai trabalhar a imagem no tamanho original ou reduzido
###############
# size_image = 'ori' # com os dados do tamanho original
size_image = 'resize' # com os dados do tamanho reduzido (512x512) ou (416x416)

#### Se for reduzido, define o tamanho
size = 640 # 416, 512 ou 640 --> yolo tem por padrão o 640

#############################
# Escolhe o tipo de treinamento - se terá ou não casos negativos
#############################

# train = 1 => 2class_TrainCovidNormal
# train = 2 => 2class_TrainCovid
# train = 3 => 1class_trainCovid
tipo_train = 3

/home/karem/Artigo_Periodico/Detector/YoloV5/1_Divide_Dados_Train_Test/


In [17]:
###############################################################################################
# Carrega o csv que contém apenas os casos negativos ou com bounding boxes delimitados
# (há casos positivos que o radiologista não delimitou os BB)
###############################################################################################

#dircsv = '/home/karem/DatasetKaggleCovid/DataCSV/CSV_Completo/CSV_CasosSemBBRemovidos.csv'
dircsv = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/CSV_CasosSemBBRemovidos.csv'

df = pd.read_csv(dircsv)
print(len(df))
df.head(5)

9589


,id,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,class_name,xmin,ymin,xmax,...,width_roi,height_roi,xmin_ori,ymin_ori,xmax_ori,ymax_ori,width_roi_ori,height_roi_ori,width_ori,height_ori
0,000a312787f2_image,5776db0cec75,0,1,0,0,typical,119.0,107.0,273.0,...,154.0,352.0,789.0,582.0,1816.0,2500.0,1026.65662,1917.30292,4256,3488
1,000a312787f2_image,5776db0cec75,0,1,0,0,typical,338.0,108.0,502.0,...,165.0,323.0,2246.0,591.0,3341.0,2353.0,1094.66162,1761.54944,4256,3488
2,000c3a3f293f_image,ff0879eb20ed,1,0,0,0,negative,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2832,2320
3,0012ff7358bc_image,9d514ce429a7,0,1,0,0,typical,142.0,50.0,324.0,...,182.0,252.0,677.0,198.0,1545.0,1198.0,867.79767,999.78214,3056,2544
4,0012ff7358bc_image,9d514ce429a7,0,1,0,0,typical,375.0,101.0,505.0,...,129.0,303.0,1793.0,403.0,2410.0,1607.0,617.02734,1204.35800,3056,2544


In [18]:
# Remove os casos atypical = outras doenças (mantém covid - typical e indeterminate; e negativos)
df2 = df.copy()
indexNames = df2[(df2['class_name'] == 'atypical')].index
df2.drop(indexNames , inplace=True)

df2 = df2.reset_index(drop=True) # reinicia o index

df2 = df2.fillna(0) # substitui NaN por zero

print(len(df2))
df2.head(5)

9052


,id,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,class_name,xmin,ymin,xmax,...,width_roi,height_roi,xmin_ori,ymin_ori,xmax_ori,ymax_ori,width_roi_ori,height_roi_ori,width_ori,height_ori
0,000a312787f2_image,5776db0cec75,0,1,0,0,typical,119.0,107.0,273.0,...,154.0,352.0,789.0,582.0,1816.0,2500.0,1026.65662,1917.30292,4256,3488
1,000a312787f2_image,5776db0cec75,0,1,0,0,typical,338.0,108.0,502.0,...,165.0,323.0,2246.0,591.0,3341.0,2353.0,1094.66162,1761.54944,4256,3488
2,000c3a3f293f_image,ff0879eb20ed,1,0,0,0,negative,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,2832,2320
3,0012ff7358bc_image,9d514ce429a7,0,1,0,0,typical,142.0,50.0,324.0,...,182.0,252.0,677.0,198.0,1545.0,1198.0,867.79767,999.78214,3056,2544
4,0012ff7358bc_image,9d514ce429a7,0,1,0,0,typical,375.0,101.0,505.0,...,129.0,303.0,1793.0,403.0,2410.0,1607.0,617.02734,1204.35800,3056,2544


In [19]:
# ## Se remover aqui os casos negativos, influencia na divisão de train/val/test
# # Remove os casos negativos 

# if tipo_train == 3:
#     df2 = df2.copy()
#     indexNames = df2[(df2['class_name'] == 'negative')].index
#     df2.drop(indexNames , inplace=True)

#     df2 = df2.reset_index(drop=True) # reinicia o index

#     df2 = df2.fillna(0) # substitui NaN por zero

#     print(len(df2))
#     df2.head(5)

In [20]:
#df3 = df2[['id', 'xmin_ori', 'ymin_ori', 'xmax_ori', 'ymax_ori']]


######### df3 = com os dados do tamanho original
if size_image == 'ori':
    ### yolo => [xmid, ymid, w, h] (normalized)
    df3 = df2[['id', 'class_name', 'xmin_ori', 'ymin_ori', 'xmax_ori', 'ymax_ori', 'width_roi_ori', \
           'height_roi_ori', 'width_ori', 'height_ori']].copy()
    #df3 = df3.groupby('id', as_index=False).agg(lambda x: x.tolist())

    # concatena linhas com o mesmo 'id'
    df3 = df3.groupby(['id', 'class_name', 'width_ori', 'height_ori'], as_index=False)['xmin_ori', \
                                        'ymin_ori', 'xmax_ori', 'ymax_ori', 'width_roi_ori', 'height_roi_ori'].agg(list)
    df3.columns = ['id', 'class_name', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax', 'width_roi', 'height_roi']
    
######### df3 = com os dados do tamanho reduzido (512x512)
else:
    df3 = df2[['id', 'class_name', 'xmin', 'ymin', 'xmax', 'ymax', 'width_roi', \
               'height_roi']].copy()

    df3['width'] = size
    df3['height'] = size


    df3 = df3.groupby(['id', 'class_name', 'width', 'height'], as_index=False)['xmin', \
                                             'ymin', 'xmax', 'ymax', 'width_roi', 'height_roi'].agg(list)

print(len(df3))
df3

5639


/tmp/ipykernel_5046/2822400746.py:25: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df3.groupby(['id', 'class_name', 'width', 'height'], as_index=False)['xmin', \


,id,class_name,width,height,xmin,ymin,xmax,ymax,width_roi,height_roi
0,000a312787f2_image,typical,640,640,"[119.0, 338.0]","[107.0, 108.0]","[273.0, 502.0]","[459.0, 432.0]","[154.0, 165.0]","[352.0, 323.0]"
1,000c3a3f293f_image,negative,640,640,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
2,0012ff7358bc_image,typical,640,640,"[142.0, 375.0]","[50.0, 101.0]","[324.0, 505.0]","[301.0, 404.0]","[182.0, 129.0]","[252.0, 303.0]"
3,001bd15d1891_image,typical,640,640,"[117.0, 484.0]","[240.0, 228.0]","[251.0, 609.0]","[493.0, 484.0]","[134.0, 124.0]","[253.0, 256.0]"
4,0022227f5adf_image,indeterminate,640,640,[390.0],[128.0],[469.0],[229.0],[79.0],[101.0]
...,...,...,...,...,...,...,...,...,...,...
5634,ffcc6edd9445_image,negative,640,640,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
5635,ffd91a2c4ca0_image,negative,640,640,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
5636,ffd9b6cf2961_image,typical,640,640,"[461.0, 499.0, 148.0]","[225.0, 491.0, 194.0]","[527.0, 555.0, 231.0]","[346.0, 573.0, 421.0]","[66.0, 56.0, 82.0]","[121.0, 82.0, 228.0]"
5637,ffdc682f7680_image,typical,640,640,"[410.0, 151.0]","[61.0, 291.0]","[635.0, 251.0]","[539.0, 433.0]","[225.0, 100.0]","[478.0, 142.0]"


In [21]:
print('Typical cases: ', df3.class_name.str.contains('typical').sum())
print('Indeterminate cases: ', df3.class_name.str.contains('indeterminate').sum())
print('Negative cases: ', df3.class_name.str.contains('negative').sum())
print('Atypical cases: ', df3.class_name.str.contains('atypical').sum())

Typical cases:  2854
Indeterminate cases:  1049
Negative cases:  1736
Atypical cases:  0


In [8]:
###############################################
### Separa os dados em treino e teste
###############################################

# 90% dos dados são utilizados para treino, 0% para validação e 10% para teste
def train_validate_test_split(df, train_percent=.9, validate_percent=.0, seed=42): # sem validação
#def train_validate_test_split(df, train_percent=.85, validate_percent=.05, seed=42): # com validação
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [9]:
# Divide o conjunto em treino e teste
train, validate, test = train_validate_test_split(df3) # mantém os casos negativos
#train, validate, test = train_validate_test_split(df4) # remove os casos negativos
train = train.reset_index(drop=True) # reinicia o index
validate = validate.reset_index(drop=True) # reinicia o index
test = test.reset_index(drop=True) # reinicia o index
print(len(train), len(validate), len(test))
train

5075 0 564


,id,class_name,width,height,xmin,ymin,xmax,ymax,width_roi,height_roi
0,611964e17dd1_image,typical,640,640,"[126.0, 398.0]","[351.0, 365.0]","[270.0, 519.0]","[498.0, 499.0]","[144.0, 122.0]","[147.0, 134.0]"
1,499ba19916d1_image,typical,640,640,"[70.0, 379.0]","[278.0, 275.0]","[261.0, 559.0]","[623.0, 604.0]","[191.0, 180.0]","[345.0, 329.0]"
2,9f126fed5074_image,indeterminate,640,640,[414.0],[343.0],[616.0],[490.0],[202.0],[147.0]
3,eda0821322f7_image,negative,640,640,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
4,041c956d9bd7_image,typical,640,640,"[501.0, 56.0]","[194.0, 156.0]","[639.0, 184.0]","[502.0, 442.0]","[138.0, 128.0]","[309.0, 285.0]"
...,...,...,...,...,...,...,...,...,...,...
5070,6c50c9903dc4_image,typical,640,640,"[74.0, 402.0]","[138.0, 188.0]","[257.0, 503.0]","[394.0, 477.0]","[183.0, 101.0]","[255.0, 288.0]"
5071,92bcb7f3b9f0_image,typical,640,640,"[95.0, 380.0]","[110.0, 264.0]","[221.0, 469.0]","[331.0, 348.0]","[127.0, 90.0]","[220.0, 84.0]"
5072,460d99af3da9_image,typical,640,640,"[134.0, 471.0]","[355.0, 307.0]","[285.0, 530.0]","[434.0, 449.0]","[150.0, 59.0]","[78.0, 141.0]"
5073,36b3565f7220_image,negative,640,640,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]


In [10]:
# se o diretório não existir, ele é criado
def createfolder(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
        
# se houver a pasta, ela é removida
import shutil
def remove_folders(path):
    if os.path.exists(path):
        shutil.rmtree(path)

In [11]:
print('Train/Val -> Typical cases: ', train.class_name.str.contains('typical').sum())
print('Train/Val -> Indeterminate cases: ', train.class_name.str.contains('indeterminate').sum())
print('Train/Val -> Negative cases: ', train.class_name.str.contains('negative').sum())
print('Train/Val -> Atypical cases: ', train.class_name.str.contains('atypical').sum())

print('_________________________________________')
print('Test -> Typical cases: ', test.class_name.str.contains('typical').sum())
print('Test -> Indeterminate cases: ', test.class_name.str.contains('indeterminate').sum())
print('Test -> Negative cases: ', test.class_name.str.contains('negative').sum())
print('Test -> Atypical cases: ', test.class_name.str.contains('atypical').sum())

Train/Val -> Typical cases:  2568
Train/Val -> Indeterminate cases:  946
Train/Val -> Negative cases:  1561
Train/Val -> Atypical cases:  0
_________________________________________
Test -> Typical cases:  286
Test -> Indeterminate cases:  103
Test -> Negative cases:  175
Test -> Atypical cases:  0


In [12]:
# Salva o dataframe de treino, validação e teste em formato txt

#folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/Folder_1/'

if tipo_train == 1:
    folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/2class_trainCovidNormal/'
elif tipo_train == 2:
    folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/2class_trainCovid/'
elif tipo_train:
    folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/1class_trainCovid/'

createfolder(folder)

train.to_csv(folder + 'trainval.csv', 
           index = False, sep=',') 
#validate.to_csv(folder + 'val.csv', 
#           index = False, sep=',')
test.to_csv(folder + 'test.csv', 
           index = False, sep=',') 

### Gerar Validação

In [13]:
def remove_negative_cases(df):
    
    indexNames = df[(df['class_name'] == 'negative')].index
    df.drop(indexNames , inplace=True)

    df = df.reset_index(drop=True) # reinicia o index

    df = df.fillna(0) # substitui NaN por zero

    # print(len(df))
    # df.head(5)
    return df

In [14]:
# Cria uma pasta contendo as imagens -> train, val e test

if size_image == 'ori':
    path_dataset = '/home/karem/DatasetKaggleCovid/Dataset/dataOriginal_PreProcessing/train/'
    
else:
#     path_dataset = '/home/karem/DatasetKaggleCovid/Dataset/dataResized_png/train/' # im 512x512
    path_dataset = '/home/karem/Artigo_Periodico/1_Dataset/2_PreProc_size640/'
#     path_dataset = diretorio + 'images416x416_pre/train/'



dest_train = diretorio + 'images_' + str(tipo_train) + '/train/'
dest_val = diretorio + 'images_' + str(tipo_train) + '/valid/'
dest_test = diretorio + 'images_' + str(tipo_train) + '/test/'

remove_folders(dest_train)
remove_folders(dest_val)
remove_folders(dest_test)

createfolder(dest_train)
createfolder(dest_val)
createfolder(dest_test)

In [15]:
########## Folder 1
# folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/Folder_1/'

import pandas as pd
#df = pd.read_csv(diretorio + 'csv_' + tipo + '/trainval.csv')
#df_test = pd.read_csv(diretorio + 'csv_' + tipo + '/test.csv')


df = pd.read_csv(folder + '/trainval.csv')
df_test = pd.read_csv(folder + '/test.csv')


# # Remove os casos negativos
# df2 = df.copy()
# indexNames = df2[(df2['class_name'] == 'negative')].index
# df2.drop(indexNames , inplace=True)
# df2 = df2.reset_index(drop=True) # reinicia o index

#df2 = df
df2 = train


print('Size train/val: ', len(df2))
print('---------------------------')


# Calcula o tamanho do dataframe de validação
size = round(len(df2) * 5 / 100) # o tamanho do df da validação será de 5% do tatal das imagens de treino
df_val = df2.iloc[:size, :]


df_val = df_val.reset_index(drop=True) # reinicia o index

print('Size val: ', len(df_val))
df_train = df2.iloc[size:, :]
df_train = df_train.reset_index(drop=True) # reinicia o index
print('Size train: ', len(df_train))
print('Size test: ', len(df_test))

Size train/val:  5075
---------------------------
Size val:  254
Size train:  4821
Size test:  564


In [16]:
#################################################################
# Remove os casos negativos apenas do conjunto de treino/val 

if tipo_train == 2: 
    df_train = remove_negative_cases(df_train)
    df_val = remove_negative_cases(df_val)
elif tipo_train == 3:
    df_train = remove_negative_cases(df_train)
    df_val = remove_negative_cases(df_val)
    df_test = remove_negative_cases(df_test)   

print('Size val: ', len(df_val))
print('Size train: ', len(df_train))
print('Size test: ', len(df_test))

Size val:  183
Size train:  3331
Size test:  389


In [17]:
# import pandas as pd
# folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/Folder_1/'
# #df = pd.read_csv(diretorio + 'csv_' + tipo + '/trainval.csv')
# df = pd.read_csv(folder + '/trainval.csv')
# df_test = pd.read_csv(diretorio + 'csv_' + tipo + '/test.csv')

# df2 = df

# # Calcula o tamanho do dataframe de validação
# size = round(len(df2) * 5 / 100) # o tamanho do df da validação será de 5% do tatal das imagens de treino

# # ################# Folder 2
# # df_val = df2.iloc[size:2*size, :]
# # df_train = pd.concat([df2.iloc[:size,:], df2.iloc[2*size:,:]])
# # folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/Folder_2/'

# # ################ Folder 3
# # df_val = df2.iloc[2*size:3*size, :]
# # df_train = pd.concat([df2.iloc[:2*size,:], df2.iloc[3*size:,:]])
# # folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/Folder_3/'


# # ################ Folder 4
# # df_val = df2.iloc[3*size:4*size, :]
# # df_train = pd.concat([df2.iloc[:3*size,:], df2.iloc[4*size:,:]])
# # folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/Folder_4/'


# # ################ Folder 5
# # df_val = df2.iloc[4*size:5*size, :]
# # df_train = pd.concat([df2.iloc[:4*size,:], df2.iloc[5*size:,:]])
# # folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/Folder_5/'

# df_val = df_val.reset_index(drop=True) # reinicia o index
# df_train = df_train.reset_index(drop=True) # reinicia o index

# print('Size train: ', len(df_train))
# print('Size val: ', len(df_val))
# print('Size test: ', len(df_test))

In [18]:
# Salva o csv de validação
df_val.to_csv(folder + 'val.csv', 
           index = False, sep=',')

# Salva o csv de treino (exclui os dados da validação do csv)
df_train.to_csv(folder + 'train.csv', 
           index = False, sep=',')

### Imagens

In [19]:
def convert_toRGB(img, dest, name):
    img = img.convert(mode='L')
    array = np.asarray(img) ## -> converte para array  
    
    # Convert to 8 bits
    array = array - np.min(array)
    array = array / np.max(array)
    array = (array * (2**8-1)).astype(np.uint8)               
    array = cv2.cvtColor(array, cv2.COLOR_GRAY2RGB)    
    
    cv2.imwrite(dest + name + '.jpg', array)

In [20]:
from PIL import Image
def save_image(df, dest):
    for i in range(len(df)):
        name = df['id'][i].replace('_image', '.png')
        im = Image.open(path_dataset + name)        
        im.save(dest + name)

In [21]:
############ Salva a imagem em formato JPG

print('início train')
save_image(df_train, dest_train)

print('início val')
save_image(df_val, dest_val)

print('início test')
save_image(df_test, dest_test)

início train
início val
início test


### Gerar txt Yolo

In [22]:
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [23]:

def convYolo(xmin, ymin, w, w_img, h, h_img):
    xcenter = (xmin + w/2) / w_img
    ycenter = (ymin + h/2) / h_img
    w = w / w_img
    h = h / h_img
    return (xcenter, ycenter, w, h)

In [24]:
import re
def createlabels(split, df):
    if size_image == 'ori':
        for i in range(len(df)): #2):
            M = []
            #f = open(split + (df['id'][i]).replace('_image', '.txt'), "w+") # cria o arquivo txt no diretório indicado e com o nome da imagem

            # percorre a quantidade de rótulos/labels
            for j in range(len(df['xmin_ori'][i])):
                l = []

                # verifica a se tem label. Se tiver recebe a classe 0 (= covid)
                # senão recebe 1 (negativo)
                if df['class_name'][i] == 'negative':
                    classes = 1
                else:
                    classes = 0

                # adiciona a classe e os rótulos na lista
                l.append(classes)
    #             l.append(df['xmin_ori'][i][j] / df['width_ori'][i])
    #             l.append(df['ymin_ori'][i][j] / df['height_ori'][i])
    #             l.append(int(df['width_roi_ori'][i][j]) / df['width_ori'][i])
    #             l.append(int(df['height_roi_ori'][i][j]) / df['height_ori'][i])

                # Calcula o Bounding Box no formato da yolo
    #             b = (df['xmin_ori'][i][j], df['ymin_ori'][i][j], df['xmax_ori'][i][j], df['ymax_ori'][i][j])
    #             bb = convert((df['width_ori'][i], df['height_ori'][i]), b)
    #             print(df['xmin_ori'][i][j])
    #             print(df['ymin_ori'][i][j])
    #             print(df['width_roi_ori'][i][j])
    #             print(df['width_ori'][i])
    #             print(df['height_roi_ori'][i][j])
    #             print(df['height_ori'][i])
                bb = convYolo(df['xmin_ori'][i][j], df['ymin_ori'][i][j], df['width_roi_ori'][i][j], \
                              df['width_ori'][i], df['height_roi_ori'][i][j], df['height_ori'][i]) 
                l.append(bb[0])
                l.append(bb[1])
                l.append(bb[2])
                l.append(bb[3])

                #print(bb[1])


                M.append(l)
                M[j][0] = np.round(M[j][0],0)

            ##l = np.array(l, int) # convert os valores da lista para inteiro
            #l = re.sub(r"[\([{})\]],", "", convert(str(l))) # convert os valores da lista para string e remove os []
            #l = l.lstrip() # remove o espaço inicial (o que está antes da classe) 

            np.savetxt(split + (df['id'][i]).replace('_image', '.txt'), M, fmt=['%d', '%s', '%s', '%s', '%s']) #fmt="%s")


            ##print(M)
            #f.write(l) # salva no txt
            #f.close()  # fecha o txt
    else:
        for i in range(len(df)): #2):
            M = []
            #f = open(split + (df['id'][i]).replace('_image', '.txt'), "w+") # cria o arquivo txt no diretório indicado e com o nome da imagem

            # percorre a quantidade de rótulos/labels
            for j in range(len(df['xmin'][i])):
                l = []

                # verifica a se tem label. Se tiver recebe a classe 0 (= covid)
                # senão recebe 1 (negativo)
                if df['class_name'][i] == 'negative':
                    classes = 1
                else:
                    classes = 0

                # adiciona a classe e os rótulos na lista
                l.append(classes)

                bb = convYolo(df['xmin'][i][j], df['ymin'][i][j], df['width_roi'][i][j], df['width'][i], \
                              df['height_roi'][i][j], df['height'][i]) 
                l.append(bb[0])
                l.append(bb[1])
                l.append(bb[2])
                l.append(bb[3])

                M.append(l)
                M[j][0] = np.round(M[j][0],0)

            np.savetxt(split + (df['id'][i]).replace('_image', '.txt'), M, fmt=['%d', '%s', '%s', '%s', '%s']) #fmt="%s")


In [25]:
folder = diretorio

# tipo = 'covid_normal'
## tipo = 'covid'
# path_train = folder + 'labels_' + tipo + '/train/'
# path_val = folder + 'labels_' + tipo + '/val/'
# path_test = folder + 'labels_' + tipo + '/test/'

path_train = folder + 'labels_' + str(tipo_train) + '/train/'
path_val = folder + 'labels_' + str(tipo_train) + '/val/'
path_test = folder + 'labels_' + str(tipo_train) + '/test/'


createfolder(path_train)
createfolder(path_val)
createfolder(path_test)


createlabels(path_train, df_train)
createlabels(path_val, df_val)
createlabels(path_test, test)

In [ ]:
# # remove as imagens do mesmo caso (retira as duplicadas)
# # porém isso remove a demarcação de outros BB
# # Esse processo foi feito para poder unir o dataframe 3 (que contém a lista de demarcações por imagem) 
# # com o tamanho da imagem (width_ori, height_ori)

# unique_df = df2.drop_duplicates(subset=['id'], keep='first')
# unique_df = unique_df.reset_index(drop=True) # reinicia o index
# print(len(unique_df))
# unique_df.head(5)

In [ ]:
# # insere as colunas de tamanho da imagem
# df4 = df3
# df4[['width_ori', 'height_ori', 'class_name']] = unique_df[['width_ori', 'height_ori', 'class_name']]
# print(len(df4))
# df4

In [ ]:
# # Remove os casos negativos 
# df5 = df4
# indexNames = df5[(df5['class_name'] == 'negative')].index
# df5.drop(indexNames , inplace=True)

# df5 = df5.reset_index(drop=True) # reinicia o index
# df5

In [ ]:
# ###############################################
# ### Separa os dados em treino e teste
# ###############################################

# # 85% dos dados são utilizados para treino, 5% para validação e 10% para teste
# def train_validate_test_split(df, train_percent=.9, validate_percent=.0, seed=None):
#     np.random.seed(seed)
#     perm = np.random.permutation(df.index)
#     m = len(df.index)
#     train_end = int(train_percent * m)
#     validate_end = int(validate_percent * m) + train_end
#     train = df.iloc[perm[:train_end]]
#     validate = df.iloc[perm[train_end:validate_end]]
#     test = df.iloc[perm[validate_end:]]
#     return train, validate, test

In [ ]:
# # Divide o conjunto em treino e teste
# train, validate, test = train_validate_test_split(df5)
# train = train.reset_index(drop=True) # reinicia o index
# validate = validate.reset_index(drop=True) # reinicia o index
# test = test.reset_index(drop=True) # reinicia o index
# print(len(train), len(validate), len(test))
# train

In [ ]:
# # Convert lista para string
# def convert(lst):
#     return  ' '.join(lst.split()) # remove os espaços extra

In [ ]:
# # se o diretório não existir, ele é criado
# def createfolder(path):
#     if not os.path.exists(path):
#         os.makedirs(path)

In [ ]:
# import re
# def createlabels(split, df):
#     for i in range(len(df)): #2):
#         M = []
#         #f = open(split + (df['id'][i]).replace('_image', '.txt'), "w+") # cria o arquivo txt no diretório indicado e com o nome da imagem
        
#         # percorre a quantidade de rótulos/labels
#         for j in range(len(df['xmin_ori'][i])):
#             l = []
            
#             # verifica a se tem label. Se tiver recebe a classe 0 (= covid)
#             # senão recebe 1 (negativo)
#             if df['xmin_ori'][i][j] == 0:
#                 classes = 1
#             else:
#                 classes = 0
#             #print(classes)
#             # adiciona a classe e os rótulos na lista
#             l.append(classes)
#             l.append(df['xmin_ori'][i][j] / df['width_ori'][i])
#             l.append(df['ymin_ori'][i][j] / df['height_ori'][i])
#             l.append(int(df['width_roi_ori'][i][j]) / df['width_ori'][i])
#             l.append(int(df['height_roi_ori'][i][j]) / df['height_ori'][i])
            
            
#             M.append(l)
#             M[j][0] = np.round(M[j][0],0)
                    
#         ##l = np.array(l, int) # convert os valores da lista para inteiro
#         #l = re.sub(r"[\([{})\]],", "", convert(str(l))) # convert os valores da lista para string e remove os []
#         #l = l.lstrip() # remove o espaço inicial (o que está antes da classe) 
        
#         np.savetxt(split + (df['id'][i]).replace('_image', '.txt'), M, fmt=['%d', '%s', '%s', '%s', '%s']) #fmt="%s")
        
                
#         ##print(M)
#         #f.write(l) # salva no txt
#         #f.close()  # fecha o txt

In [ ]:
# !rm -fr /home/karem/.local/share/Trash/files/*